In [ ]:
from src.training import *
from src.pipeline.loaders import collate_numpy_matrices_without_conditions
from src.pipeline.datasets import ChunkedDiskCachedDatasetWrapper
import warnings
warnings.filterwarnings("ignore", message="Unknown entity 'ee-infinity-loader'")

has_been_cached = True  # Used to avoid reprocessing our dataset once we've done it once.

# Added something to speed up subsequent training.

dims = (20,20)
if not has_been_cached:
    rv = 5
    datalist = prepare_dataset(dims=dims, repr_version=rv)
    rotational_datalist = AugmentedListDataset(*datalist)
    cached_dataset = ChunkedDiskCachedDatasetWrapper(rotational_datalist,
                                                     force_rebuild=not has_been_cached,
                                                     cache_dir='dataset_cache')
    # TODO don't i have to save the cache
else:
    cached_dataset = ChunkedDiskCachedDatasetWrapper.from_cache(cache_dir='dataset_cache')

dataloader = DataLoader(
    cached_dataset,
    batch_size=32, 
    collate_fn=collate_numpy_matrices_without_conditions
)
train_dataloader, val_dataloader = split_dataloader(dataloader, val_split=0.4)

input_size = dims[0]

In [ ]:
from src.model import train_model, BinaryMatrixTransformCNN


model = BinaryMatrixTransformCNN(matrix_size=input_size)

train_model(model, train_loader=train_dataloader, val_loader=val_dataloader,
            num_epochs=50, device='mps')

In [ ]:
from src.model import BinaryMatrixTransformCNN
from src.visualization import visualize_manychannel_matrices
from src.pipeline.loaders import collate_numpy_matrices_without_conditions
from torch.utils.data import DataLoader
import torch

input_size=20

model = BinaryMatrixTransformCNN(matrix_size=input_size)
model.load_state_dict(torch.load('best_model.pt'))

dataloader = DataLoader(
    cached_dataset,
    batch_size=1, 
    collate_fn=collate_numpy_matrices_without_conditions
)

L = len(list(dataloader))//4
X, Y = list(dataloader)[983 + 2*L]
O = model.predict(X)
visualize_manychannel_matrices(X[0], Y[0], O[0])